In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import seaborn as sns
cudnn.benchmark = True
import sklearn.metrics as metrics
from PIL import Image
from prettytable import PrettyTable
import cv2
from tqdm import tqdm


In [2]:
output_path = "Z://RR/Final/report_work/results/Trained_on_FairFace_Testing/UTKFace_tests"

model_name = 'ResNet34_v21_5'
model_path = F"Z://RR/Final/report_work/saved_models/FairFace_All_Races_Testing/{model_name}.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def show_image(file_path):
	# full_path = file_prefix + '/'+file_path
	img = cv2.imread(file_path)[:,:,::-1]
	plt.imshow(img)
	plt.show()
 
dataframe_path = 'Z://RR/UTKFace/labels/processed_aligned_UTKFace.csv'
UTKFace_df = pd.read_csv(dataframe_path).rename(columns={"name":"file"})

UTKFace_df.drop(['Unnamed: 0','label'],axis =1,inplace = True)
UTKFace_df = UTKFace_df[UTKFace_df['ethnicity'] != 'Others']
UTKFace_df['prediction'] = ""

In [3]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
	'test': transforms.Compose([ #compose several transforms together
		transforms.Resize(224),
		transforms.RandomHorizontalFlip(),
		transforms.ToTensor(),
		# transforms.Normalize(mean, std)
	]),
}

In [4]:
def load_checkpoint(model, load_path):
	if torch.cuda.is_available():
		
		checkpoint = torch.load(load_path)
	else:
		checkpoint = torch.load(load_path,map_location=torch.device('cpu'))
		
	model.load_state_dict(checkpoint['model_state_dict'])

	return model


def get_model_data(class_names):
	weights = models.ResNet34_Weights.DEFAULT
	model_ft = models.resnet34(weights = weights)
	num_ftrs = model_ft.fc.in_features
 
	model_ft.fc = nn.Linear(num_ftrs,len(class_names))
 
	model_ft= load_checkpoint(model_ft, model_path)
 
	model_ft = model_ft.to(device)
	model_ft.eval()
 
	return model_ft

def get_pred(model,file):
	img = Image.open(file)
	input = data_transforms['test'](img)
	input = input.unsqueeze(0)
	input = input.to(device)
	with torch.set_grad_enabled(False):
		output = model(input)
		_, preds = torch.max(output, 1)
	
	return preds[0].item() 
	

In [5]:
class_names = UTKFace_df.age_group.value_counts().reset_index(name = "count").rename(columns={"index":"age"})['age']
model = get_model_data(class_names)

In [6]:

for index in tqdm(UTKFace_df.index):
    
	# record = UTKFace_df.iloc[index]
	path = UTKFace_df.at[index,'file']
	if os.path.isfile(path):
		class_pred = get_pred(model,path)
	
		UTKFace_df.at[index,'prediction'] = class_pred

100%|██████████| 22016/22016 [41:49<00:00,  8.77it/s] 


In [7]:
UTKFace_df = UTKFace_df[UTKFace_df['prediction'].notna()]
UTKFace_df = UTKFace_df[UTKFace_df.prediction.map(type)==int]

age_dist = UTKFace_df.age_group.value_counts().reset_index(name = "count").rename(columns={"index":"age"})
age_dist.sort_values(by = ['age'],inplace=True)
age_dist = age_dist.reset_index(drop=True)
age_groups = age_dist['age']

In [12]:
def convert(pred):
    for i,group in enumerate(age_groups):
        if group == pred:
            return i

In [13]:
UTKFace_df['pred_age_group'] = UTKFace_df['prediction'].apply(lambda x: age_groups[x])
UTKFace_df['true'] = UTKFace_df['age_group'].apply(lambda x: convert(x))


In [19]:
UTKFace_df.head()

,file,age,gender,ethnicity,age_group,new_ethnicity,prediction,pred_age_group,true
0,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,0,white,more_than_70,white,8,more_than_70,8
1,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,0,white,more_than_70,white,8,more_than_70,8
2,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,1,white,more_than_70,white,8,more_than_70,8
3,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,1,white,more_than_70,white,8,more_than_70,8
4,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,1,white,more_than_70,white,8,more_than_70,8


In [11]:
UTKFace_df.to_csv(output_path+'/UTKFace_results.csv')

In [32]:
def convert_one_off(t,pred):
	if t == pred:
		return t

	if pred != 0:
		if (pred-1) == t:
			return t

	if pred != 8:
		if (pred+1) == t:
			return t
	
	return pred

In [40]:
output_path

'Z://RR/Final/report_work/results/Trained_on_FairFace_Testing/FairFace_tests'

In [36]:

UTKFace_df['pred_one_off'] = UTKFace_df.apply(lambda x: convert_one_off(x['true'],x['prediction']),axis=1)
# UTKFace_df.assign(pred_one_off = lambda x: convert_one_off(UTKFace_df['true'],UTKFace_df['prediction']))


In [37]:
UTKFace_df.head()

,file,age,gender,ethnicity,age_group,new_ethnicity,prediction,pred_age_group,true,pred_one_off
0,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,0,white,more_than_70,white,8,more_than_70,8,8
1,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,0,white,more_than_70,white,8,more_than_70,8,8
2,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,1,white,more_than_70,white,8,more_than_70,8,8
3,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,1,white,more_than_70,white,8,more_than_70,8,8
4,Z://RR/UTKFace/utkface_aligned_cropped/UTKFace...,100,1,white,more_than_70,white,8,more_than_70,8,8


In [39]:
UTKFace_df.to_csv(output_path+'/UTKFace_results_one_off.csv')
